Import lib

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
from PIL import Image
from torchsummary import summary
from torchvision.io import read_image 
from pathlib import Path
import numpy as np


In [ ]:
train_path = "FER-13/train"
test_path = "FER-13/test"


In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    #transforms.ToDtype(torch.float32, scale=True),
])

In [ ]:

train_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=test_path, transform=transform)

In [ ]:
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
batch_size = 256
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
classes = os.listdir(train_path)
fig, axs = plt.subplots(1, 7, figsize=(15, 5))
for i, class_name in enumerate(classes):
    img_path = os.path.join(train_path, class_name, os.listdir(os.path.join(train_path, class_name))[0])
    img = Image.open(img_path)
    axs[i].imshow(img, cmap='gray')
    axs[i].set_title(class_name)
    axs[i].axis('off')
plt.show()

In [ ]:
r50_weights = models.ResNet50_Weights.DEFAULT
r50 = models.resnet50(weights=r50_weights)
#r50.eval()

# Define ResNet50 with modified first layer for grayscale
class gsr50(nn.Module):
    def __init__(self, num_classes=7):
        super(gsr50, self).__init__()

        # Replacing first conv layer for grayscale image
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

        # Keeping rest of the ResNet50 architecture same
        self.layer1 = nn.Sequential(*list(r50.layer1))
        self.layer2 = nn.Sequential(*list(r50.layer2))
        self.layer3 = nn.Sequential(*list(r50.layer3))
        self.layer4 = nn.Sequential(*list(r50.layer4))

        # Modifying last fc layer for 7-class classification
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)  # Adjust input size based on your architecture

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

r50_new = gsr50(num_classes=7)
#r50_new.eval()


In [ ]:
#mv3 = models.mobilenet_v3_large(pretrained=True)


In [ ]:
#v16 = models.vgg16(pretrained=True)

Training models

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(r50_new.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data,1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct / total
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
    return train_losses,val_losses,train_accuracies,val_accuracies

In [ ]:
n_epochs = 20
train_losses,val_losses,train_accuracies,val_accuracies = train_model(r50_new, train_loader, val_loader, criterion,optimizer, num_epochs=n_epochs)